<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Отворен курс за машинно обучение

Автор: [Егор Полусмак](https://www.linkedin.com/in/egor-polusmak/). Преведено и редактирано от [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/). Този материал е предмет на правилата и условията на лиценза [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Безплатното използване е разрешено за всякакви нетърговски цели.

# <center>Тема 9. Анализ на времеви редове в Python</center>
## <center>Част 2. Предсказване на бъдещето с Facebook Prophet</center>

Прогнозирането на времеви редове намира широко приложение в анализа на данни. Това са само някои от възможните прогнози за бъдещи тенденции, които могат да бъдат полезни:
- Броят сървъри, от които една онлайн услуга ще се нуждае през следващата година.
- Търсенето на хранителен продукт в супермаркет за даден ден.
- Утрешната цена на затваряне на търгуем финансов актив.

За друг пример, можем да направим прогноза за представянето на някой екип и след това да го използваме като базова линия: първо да зададем цели за екипа и след това да измерим действителното представяне на екипа спрямо базовата линия.

Има доста различни методи за прогнозиране на бъдещи тенденции, например [ARIMA](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average), [ARCH](https://en.wikipedia.org/wiki/ Авторегресивна_условна_хетероскедастичност), [регресивни модели](https://en.wikipedia.org/wiki/Autoregressive_model), [невронни мрежи](https://medium.com/machine-learning-world/neural-networks-for-algorithmic- trading-1-2-correct-time-series-forecasting-backtesting-9776bfd9e589).

В тази статия ще разгледаме [Prophet](https://facebook.github.io/prophet/), библиотека за прогнозиране на времеви редове, пусната от Facebook и с отворен код на 23 февруари 2017 г. Ние също ще я изпробваме в проблема за прогнозиране на дневния брой публикации, публикувани в Medium.

## Описание на статията

1. Въведение
2. Моделът за прогнозиране на Пророка
3. Практикувайте с Пророка
    * 3.1 Инсталиране в Python
    * 3.2 Набор от данни
    * 3.3 Проучвателен визуален анализ
    * 3.4 Изготвяне на прогноза
    * 3.5 Оценка на качеството на прогнозата
    * 3.6 Визуализация
4. Трансформация на Бокс-Кокс
5. Обобщение
6. Използвана литература

## 1. Въведение

Според [статията](https://research.fb.com/prophet-forecasting-at-scale/) във Facebook Research, Prophet първоначално е разработен с цел създаване на висококачествени бизнес прогнози. Тази библиотека се опитва да се справи със следните трудности, общи за много бизнес времеви редове:
- Сезонни ефекти, причинени от човешкото поведение: седмични, месечни и годишни цикли, спадове и пикове на официални празници.
- Промени в тенденцията поради нови продукти и пазарни събития.
- Отклонения.

Авторите твърдят, че дори и с настройките по подразбиране, в много случаи тяхната библиотека произвежда прогнози, толкова точни, колкото тези, предоставени от опитни анализатори.

Освен това, Prophet има редица интуитивни и лесно интерпретируеми персонализации, които позволяват постепенно подобряване на качеството на модела за прогнозиране. Това, което е особено важно, тези параметри са доста разбираеми дори за неспециалисти в анализа на времеви редове, което е област на науката за данни, изискваща определени умения и опит.

Между другото, оригиналната статия се нарича „Прогнозиране в мащаб“, но не става въпрос за мащаба в „обичайния“ смисъл, който се отнася до изчислителни и инфраструктурни проблеми на голям брой работещи програми. Според авторите, Prophet трябва да се мащабира добре в следните 3 области:
- Достъпност до широка аудитория от анализатори, вероятно без задълбочени познания по времеви редове.
- Приложимост към широк кръг от различни проблеми с прогнозирането.
- Автоматизирана оценка на ефективността на голям брой прогнози, включително маркиране на потенциални проблеми за последващата им проверка от анализатора.

## 2. Моделът за прогнозиране на Пророка

Сега нека разгледаме по-отблизо как работи Prophet. По своята същност тази библиотека използва [допълнителния регресионен модел](https://en.wikipedia.org/wiki/Additive_model) $y(t)$, включващ следните компоненти:

$$y(t) = g(t) + s(t) + h(t) + \epsilon_{t},$$

където:
* Тенденцията $g(t)$ моделира непериодични промени.
* Сезонността $s(t)$ представлява периодични промени.
* Празничен компонент $h(t)$ предоставя информация за празници и събития.

По-долу ще разгледаме някои важни свойства на тези компоненти на модела.

### Тенденция

Библиотеката Prophet прилага два възможни модела на тенденции за $g(t)$.

Първият се нарича *Нелинеен, насищащ растеж*. Той е представен под формата на [логистичен модел на растеж](https://en.wikipedia.org/wiki/Logistic_function):

$$g(t) = \frac{C}{1+e^{-k(t - m)}},$$

където:
* $C$ е товароносимостта (това е максималната стойност на кривата).
* $k$ е скоростта на растеж (която представлява "стръмността" на кривата).
* $m$ е параметър за отместване.

Това логистично уравнение позволява моделиране на нелинеен растеж с насищане, т.е. когато скоростта на нарастване на стойност намалява с нейния растеж. Един от типичните примери би бил представянето на нарастването на аудиторията на приложение или уебсайт.

Всъщност $C$ и $k$ не са непременно константи и могат да варират с времето. Prophet поддържа както автоматична, така и ръчна настройка на тяхната променливост. Библиотеката може сама да избере оптимални точки на промени в тенденцията чрез монтиране на предоставените исторически данни.

Освен това Prophet позволява на анализаторите ръчно да задават точки на промяна на стойностите на темпа на растеж и капацитета в различни моменти от време. Например, анализаторите може да имат представа за дати на минали версии, които значително са повлияли на някои ключови продуктови индикатори.

Вторият трендов модел е прост *частично линеен модел* с постоянна скорост на растеж. Най-подходящ е за проблеми без насищащ растеж.

### Сезонност

Сезонният компонент $s(t)$ предоставя гъвкав модел на периодични промени поради седмична и годишна сезонност.

Седмичните сезонни данни се моделират с фиктивни променливи. Добавени са шест нови променливи: `понеделник`, `вторник`, `сряда`, `четвъртък`, `петък`, `събота`, които приемат стойности 0 или 1 в зависимост от деня от седмицата. Характеристиката „неделя“ не е добавена, защото би била линейна комбинация от останалите дни от седмицата и този факт би имал неблагоприятен ефект върху модела.

Моделът на годишната сезонност в Prophet разчита на редове на Фурие.

От [версия 0.2](https://github.com/facebook/prophet) можете също да използвате *поддневни времеви серии* и да правите *поддневни прогнози*, както и да използвате новата функция *ежедневна сезонност*.

### Празници и събития

Компонентът $h(t)$ представлява предвидими необичайни дни от годината, включително тези с нередовни графици, например Черни петъци.

За да използва тази функция, анализаторът трябва да предостави персонализиран списък от събития.

### Грешка

Терминът за грешка $\epsilon(t)$ представлява информация, която не е отразена в модела. Обикновено се моделира като нормално разпределен шум.

### Пророк Бенчмаркинг

За подробно описание на модела и алгоритмите зад Prophet вижте статията [„Прогнозиране в мащаб“](https://peerj.com/preprints/3190/) от Sean J. Taylor и Benjamin Letham.

Авторите също сравняват своята библиотека с няколко други метода за прогнозиране на времеви редове. Те използваха [средна абсолютна процентна грешка (MAPE)] (https://en.wikipedia.org/wiki/Mean_absolute_percentage_error) като мярка за точност на прогнозата. В това изследване Prophet показа значително по-ниска грешка при прогнозиране от другите модели.

<img src="../../img/topic9_benchmarking_prophet.png" />

Нека разгледаме по-отблизо как е измерено качеството на прогнозиране в статията. За да направим това, ще ни трябва формулата за средна абсолютна процентна грешка.

Нека $y_{i}$ е *действителната (историческа) стойност* и $\hat{y}_{i}$ е *прогнозната стойност*, дадена от нашия модел.

Тогава $e_{i} = y_{i} - \hat{y}_{i}$ е *грешката на прогнозата* и $p_{i} =\frac{\displaystyle e_{i}}{\displaystyle y_{ i}}$ е *относителната грешка в прогнозата*.

Ние определяме

$$MAPE = mean\big(\left |p_{i} \right |\big)$$

MAPE се използва широко като мярка за точност на прогнозиране, тъй като изразява грешката като процент и по този начин може да се използва в оценките на модела на различни набори от данни.

В допълнение, когато се оценява алгоритъм за прогнозиране, може да се окаже полезно да се изчисли [MAE (средна абсолютна грешка)](https://en.wikipedia.org/wiki/Mean_absolute_error), за да имате картина на грешките в абсолютни числа. Използвайки предварително определени компоненти, неговото уравнение ще бъде

$$MAE = mean\big(\left |e_{i}\right |\big)$$

Няколко думи за алгоритмите, с които беше сравнен Prophet. Повечето от тях са доста прости и често се използват като основа за други модели:
* „наивен“ е опростен подход за прогнозиране, при който предвиждаме всички бъдещи стойности, разчитайки единствено на наблюдението в последния наличен момент от време.
* `snaive` (seasonal naive) е модел, който прави постоянни прогнози, като взема предвид информацията за сезонността. Например, в случай на седмични сезонни данни за всеки бъдещ понеделник, бихме предвидили стойността от последния понеделник, а за всички бъдещи вторници ще използваме стойността от последния вторник и т.н.
* `mean` използва средната стойност на данните като прогноза.
* `arima` означава *Autoregressive Integrated Moving Average*, вижте [Wikipedia](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average) за подробности.
* `ets` означава *Exponential Smoothing*, вижте [Wikipedia](https://en.wikipedia.org/wiki/Exponential_smoothing) за повече.

## 3. Практикувайте с Facebook Prophet

### 3.1 Инсталиране в Python

Първо, трябва да инсталирате библиотеката. Prophet е наличен за Python и R. Изборът ще зависи от вашите лични предпочитания и изисквания на проекта. По-нататък в тази статия ще използваме Python.

В Python можете да инсталирате Prophet с помощта на PyPI:
```
$ pip инсталирайте fbprophet
```

В R можете да намерите съответния CRAN пакет. Обърнете се към [документацията](https://facebookincubator.github.io/prophet/docs/installation.html) за подробности.

Нека импортираме модулите, които ще ни трябват, и инициализираме нашата среда:

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

%matplotlib inline

### 3.2 Набор от данни

Ще предвидим дневния брой публикации, публикувани в [Medium](https://medium.com/).

Първо зареждаме нашия набор от данни.

In [ ]:
df = pd.read_csv("../../data/medium_posts.csv.zip", sep="\t")

След това пропускаме всички колони с изключение на „published“ и „url“. Първото съответства на времевото измерение, докато второто уникално идентифицира публикация чрез нейния URL адрес. По пътя се отърваваме от възможни дубликати и липсващи стойности в данните:


In [ ]:
df = df[["published", "url"]].dropna().drop_duplicates()

След това трябва да преобразуваме `published` във формат за дата и час, защото по подразбиране `pandas` третира това поле като стойност на низ.


In [ ]:
df["published"] = pd.to_datetime(df["published"])

Нека сортираме рамката с данни по време и да разгледаме какво имаме:


In [ ]:
df.sort_values(by=["published"]).head(n=3)

Датата на публично пускане на Medium беше 15 август 2012 г. Но, както можете да видите от данните по-горе, има поне няколко реда с много по-ранни дати на публикуване. Те някак си се появиха в нашия набор от данни, но едва ли са законни. Просто ще изрежем нашата времева поредица, за да запазим само онези редове, които попадат в периода от 15 август 2012 г. до 25 юни 2017 г.:

In [ ]:
df = df[
    (df["published"] > "2012-08-15") & (df["published"] < "2017-06-26")
].sort_values(by=["published"])
df.head(n=3)

In [ ]:
df.tail(n=3)

Тъй като ще предвидим броя на публикуваните публикации, ще обобщим и преброим уникалните публикации във всеки даден момент от време. Ще кръстим съответната нова колона „публикации“:

In [ ]:
aggr_df = df.groupby("published")[["url"]].count()
aggr_df.columns = ["posts"]

В тази практика се интересуваме от броя публикации **на ден**. Но в този момент всички наши данни са разделени на нередовни интервали от време, които са по-малки от един ден. Това се нарича *поддневен времеви ред*. За да го видите, нека отпечатаме първите 3 реда:


In [ ]:
aggr_df.head(n=3)

За да коригираме това, трябва да обединим броя на публикациите по „кошчета“ с размер на датата. В анализа на времеви редове този процес се нарича *повторно вземане на проби*. И ако *намалим* честотата на дискретизация на данните, това често се нарича *намаляване на дискретизацията*.

За щастие, `pandas` има вградена функционалност за тази задача. Ние ще преобразуваме нашия времеви индекс до 1-дневни кошчета:

In [ ]:
daily_df = aggr_df.resample("D").apply(sum)
daily_df.head(n=3)

### 3.3 Проучвателен визуален анализ

Както винаги, може да е полезно и поучително да разгледате графично представяне на вашите данни.

Ще създадем диаграма на времеви серии за целия времеви диапазон. Показването на данни за толкова дълъг период от време може да даде указания за сезонността и забележимите необичайни отклонения.

Първо импортираме и инициализираме библиотеката `Plotly`, която позволява създаването на красиви интерактивни графики:

In [ ]:
from plotly import graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# Initialize plotly
init_notebook_mode(connected=True)

Ние също така дефинираме помощна функция, която ще начертае нашите кадри с данни в цялата статия:

In [ ]:
def plotly_df(df, title=""):
    """Visualize all the dataframe columns as line plots."""
    common_kw = dict(x=df.index, mode="lines")
    data = [go.Scatter(y=df[c], name=c, **common_kw) for c in df.columns]
    layout = dict(title=title)
    fig = dict(data=data, layout=layout)
    iplot(fig, show_link=False)

Нека се опитаме да начертаем нашия набор от данни *както е*:

In [ ]:
plotly_df(daily_df, title="Posts on Medium (daily)")

Данните с висока честота могат да бъдат доста трудни за анализ. Дори с възможността за приближаване, осигурена от `Plotly`, е трудно да се направи извод за нещо смислено от тази диаграма, освен изпъкналата възходяща и ускоряваща се тенденция.

За да намалим шума, ще направим повторно отброяване на публикациите до седмични кошчета. Освен *binning*, други възможни техники за намаляване на шума включват [Moving-Average Smoothing](https://en.wikipedia.org/wiki/Moving_average) и [Exponential Smoothing](https://en.wikipedia.org/wiki /Exponential_smoothing), между другото.

Ние запазваме нашата рамка с данни с намалена дискретизация в отделна променлива, защото по-нататък в тази практика ще работим само с дневни серии:

In [ ]:
weekly_df = daily_df.resample("W").apply(sum)

Накрая начертаваме резултата:

In [ ]:
plotly_df(weekly_df, title="Posts on Medium (weekly)")

Тази диаграма с намалена дискретизация се оказва малко по-добра за възприятието на анализатора.

Една от най-полезните функции, които `Plotly` предоставя, е способността бързо да се потопите в различни периоди от времевата линия, за да разберете по-добре данните и да намерите визуални улики за възможни тенденции, периодични и нередовни ефекти.

Например, увеличаването на няколко последователни години ни показва времеви точки, съответстващи на коледните празници, които силно влияят на човешкото поведение.

Сега ще пропуснем първите няколко години на наблюдения, до 2015 г. Първо, те няма да допринесат много за прогнозното качество през 2017 г. Второ, тези първи години, с много малък брой публикации на ден, са вероятно ще увеличи шума в нашите прогнози, тъй като моделът ще бъде принуден да съобрази тези необичайни исторически данни заедно с по-подходящи и показателни данни от последните години.

In [ ]:
daily_df = daily_df.loc[daily_df.index >= "2015-01-01"]
daily_df.head(n=3)

За да обобщим, от визуален анализ можем да видим, че нашият набор от данни е нестационарен с видима нарастваща тенденция. Той също така демонстрира седмична и годишна сезонност и брой необичайни дни във всяка година.


### 3.4 Изготвяне на прогноза

API на Prophet е много подобен на този, който можете да намерите в `sklearn`. Първо създаваме модел, след това извикваме метода „fit“ и накрая правим прогноза. Входът за метода `fit` е `DataFrame` с две колони:
* `ds` (клеймо за дата) трябва да е от тип `date` или `datetime`.
* `y` е числова стойност, която искаме да предвидим.

За да започнем, ще импортираме библиотеката и ще заглушим маловажните диагностични съобщения:

In [ ]:
import logging

from fbprophet import Prophet

logging.getLogger().setLevel(logging.ERROR)

Нека преобразуваме нашата рамка от данни във формата, изискван от Prophet:

In [ ]:
df = daily_df.reset_index()
df.columns = ["ds", "y"]
# converting timezones (issue https://github.com/facebook/prophet/issues/831)
df["ds"] = df["ds"].dt.tz_convert(None)
df.tail(n=3)

Авторите на библиотеката обикновено съветват да се правят прогнози въз основа на поне няколко месеца, в идеалния случай повече от година исторически данни. За щастие, в нашия случай разполагаме с повече от няколко години данни, които да отговарят на модела.

За да измерим качеството на нашата прогноза, трябва да разделим нашия набор от данни на *историческата част*, която е първият и най-голям отрязък от нашите данни, и *прогнозната част*, която ще бъде разположена в края на времевата линия. Ще премахнем последния месец от набора от данни, за да го използваме по-късно като цел за прогнозиране:

In [ ]:
prediction_size = 30
train_df = df[:-prediction_size]
train_df.tail(n=3)

Сега трябва да създадем нов обект `Prophet`. Тук можем да предадем параметрите на модела в конструктора. Но в тази статия ще използваме настройките по подразбиране. След това обучаваме нашия модел, като извикваме неговия метод `fit` на нашия набор от данни за обучение:


In [ ]:
m = Prophet()
m.fit(train_df);

Използвайки помощния метод `Prophet.make_future_dataframe`, ние създаваме рамка от данни, която ще съдържа всички дати от историята и също ще се простира в бъдещето за тези 30 дни, които сме пропуснали преди.


In [ ]:
future = m.make_future_dataframe(periods=prediction_size)
future.tail(n=3)

Ние прогнозираме стойности с `Prophet`, като предаваме датите, за които искаме да създадем прогноза. Ако предоставим и историческите дати (както в нашия случай), тогава в допълнение към прогнозата ще получим вписване в извадката за историята. Нека извикаме метода `predict` на модела с нашия `бъдещ` кадър от данни като вход:


In [ ]:
forecast = m.predict(future)
forecast.tail(n=3)

В резултантната рамка с данни можете да видите много колони, характеризиращи прогнозата, включително компоненти за тенденция и сезонност, както и техните доверителни интервали. Самата прогноза се съхранява в колона `yhat`.

Библиотеката Prophet има свои собствени вградени инструменти за визуализация, които ни позволяват бързо да оценим резултата.

Първо, има метод, наречен `Prophet.plot`, който начертава всички точки от прогнозата:

In [ ]:
m.plot(forecast);

Тази диаграма не изглежда много информативна. Единственото окончателно заключение, което можем да направим тук, е, че моделът третира много от точките с данни като извънредни стойности.

Втората функция `Prophet.plot_components` може да бъде много по-полезна в нашия случай. Това ни позволява да наблюдаваме отделно различни компоненти на модела: тенденция, годишна и седмична сезонност. Освен това, ако предоставите информация за празници и събития на вашия модел, те също ще бъдат показани в този график.

Нека да го изпробваме:

In [ ]:
m.plot_components(forecast);

Както можете да видите от графиката на тенденциите, Prophet свърши добра работа, като приспособи ускорения растеж на новите публикации в края на 2016 г. Графиката на седмичната сезонност води до заключението, че обикновено има по-малко нови публикации в събота и неделя, отколкото в останалите дни от седмицата. В годишната графика на сезонността има забележим спад на Коледа.

### 3.5 Оценка на качеството на прогнозата

Нека да оценим качеството на алгоритъма, като изчислим показателите за грешки за последните 30 дни, които прогнозирахме. За целта ще ни трябват наблюденията $y_i$ и съответните прогнозирани стойности $\hat{y}_i$.

Нека да разгледаме обекта „прогноза“, създаден от библиотеката за нас:

In [ ]:
print(", ".join(forecast.columns))

Можем да видим, че тази рамка от данни съдържа цялата информация, от която се нуждаем, с изключение на историческите стойности. Трябва да обединим обекта „прогноза“ с действителните стойности „y“ от оригиналния набор от данни „df“. За целта ще дефинираме помощна функция, която ще използваме повторно по-късно:


In [ ]:
def make_comparison_dataframe(historical, forecast):
   """съединете историята с прогнозата.
    
       Полученият набор от данни ще съдържа колони „yhat“, „yhat_lower“, „yhat_upper“ и „y“.
    """
    return forecast.set_index("ds")[["yhat", "yhat_lower", "yhat_upper"]].join(
        historical.set_index("ds")
    )

Нека приложим тази функция към последната ни прогноза:


In [ ]:
cmp_df = make_comparison_dataframe(df, forecast)
cmp_df.tail(n=3)

Също така ще дефинираме помощна функция, която ще използваме, за да преценим качеството на нашите прогнози с MAPE и MAE мерки за грешка:

In [ ]:
def calculate_forecast_errors(df, prediction_size):
    """Изчислете MAPE и MAE на прогнозата.
    
       Аргументи:
           df: обединен набор от данни с колони „y“ и „yhat“.
           prediction_size: брой дни в края за прогнозиране.
    """

    # Направи копие
    df = df.copy()

    # Сега изчисляваме стойностите на e_i и p_i според формулите, дадени в статията по-горе.
    df["e"] = df["y"] - df["yhat"]
    df["p"] = 100 * df["e"] / df["y"]

    # Спомнете си, че запазихме стойностите от последните `prediction_size` дни
    # за да ги предвидим и измерим качеството на модела.
    # Сега изрежете частта от данните, за които направихме нашата прогноза.
    predicted_part = df[-prediction_size:]

    # Дефинирайте функцията, която усреднява стойностите на абсолютната грешка върху предвидената част.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))

    # Сега можем да изчислим MAPE и MAE и да върнем резултантния речник на грешките.
    return {"MAPE": error_mean("p"), "MAE": error_mean("e")}

Нека използваме нашата функция:

In [ ]:
for err_name, err_value in calculate_forecast_errors(cmp_df, prediction_size).items():
    print(err_name, err_value)

В резултат на това относителната грешка на нашата прогноза (MAPE) е около 22,6%, а средно нашият модел греши с ~70 публикации (MAE).

### 3.6 Визуализация

Нека създадем наша собствена визуализация на модела, изграден от Prophet. Той ще включва действителните стойности, прогнозата и доверителните интервали.

Първо, ще начертаем данните за по-кратък период от време, за да направим точките от данни по-лесни за разграничаване. Второ, ще покажем производителността на модела само за периода, който прогнозирахме, тоест последните 30 дни. Изглежда, че тези две мерки трябва да ни дадат по-четлив график.

Трето, ще използваме `Plotly`, за да направим нашата диаграма интерактивна, което е чудесно за изследване.

Ще дефинираме персонализирана помощна функция `show_forecast` и ще я извикаме (за повече информация как работи, моля, вижте коментарите в кода и [документацията](https://plot.ly/python/)):

In [ ]:
def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""

    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode="lines")
        args.update(kwargs)
        return go.Scatter(**args)

    lower_bound = create_go(
        "Lower Bound",
        "yhat_lower",
        num_predictions,
        line=dict(width=0),
        marker=dict(color="gray"),
    )
    upper_bound = create_go(
        "Upper Bound",
        "yhat_upper",
        num_predictions,
        line=dict(width=0),
        marker=dict(color="gray"),
        fillcolor="rgba(68, 68, 68, 0.3)",
        fill="tonexty",
    )
    forecast = create_go(
        "Forecast", "yhat", num_predictions, line=dict(color="rgb(31, 119, 180)")
    )
    actual = create_go("Actual", "y", num_values, marker=dict(color="red"))

    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title="Posts"), title=title, showlegend=False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)


show_forecast(cmp_df, prediction_size, 100, "New posts on Medium")

На пръв поглед предвиждането на средните стойности от нашия модел изглежда разумно. Високата стойност на MAPE, която получихме по-горе, може да се обясни с факта, че моделът не успя да хване нарастващата амплитуда от пик до пик на слаба сезонност.

Също така можем да заключим от графиката по-горе, че много от действителните стойности са извън доверителния интервал. Prophet може да не е подходящ за времеви редове с нестабилна вариация, поне когато се използват настройките по подразбиране. Ще се опитаме да поправим това, като приложим трансформация към нашите данни.

## 4. Box-Cox Transformation

Досега използвахме Prophet с настройките по подразбиране и оригиналните данни. Ще оставим само параметрите на модела. Но въпреки това все още имаме място за подобрение. В този раздел ще приложим [трансформацията на Бокс–Кокс](http://onlinestatbook.com/2/transformations/box-cox.html) към нашата оригинална серия. Да видим докъде ще ни доведе.

Няколко думи за тази трансформация. Това е монотонна трансформация на данни, която може да се използва за стабилизиране на дисперсията. Ще използваме еднопараметричната трансформация на Box–Cox, която се дефинира от следния израз:


$$
\begin{equation}
  boxcox^{(\lambda)}(y_{i}) = \begin{cases}
    \frac{\displaystyle y_{i}^{\lambda} - 1}{\displaystyle \lambda} &, \text{if $\lambda \neq 0$}.\\
    ln(y_{i}) &, \text{if $\lambda = 0$}.
  \end{cases}
\end{equation}
$$

Ще трябва да приложим обратното на тази функция, за да можем да възстановим оригиналния мащаб на данните. Лесно е да се види, че обратното се дефинира като:

$$
\begin{equation}
  invboxcox^{(\lambda)}(y_{i}) = \begin{cases}
    e^{\left (\frac{\displaystyle ln(\lambda y_{i} + 1)}{\displaystyle \lambda} \right )} &, \text{if $\lambda \neq 0$}.\\
    e^{y_{i}} &, \text{if $\lambda = 0$}.
  \end{cases}
\end{equation}
$$

Съответната функция в Python е реализирана по следния начин:


In [ ]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

Първо подготвяме нашия набор от данни, като задаваме неговия индекс:

In [ ]:
train_df2 = train_df.copy().set_index("ds")

След това прилагаме функцията `stats.boxcox` от `Scipy`, която прилага трансформацията на Box–Cox. В нашия случай ще върне две стойности. Първата е трансформираната серия, а втората е намерената стойност на $\lambda$, която е оптимална по отношение на максималната логаритмична вероятност:

In [ ]:
train_df2["y"], lambda_prophet = stats.boxcox(train_df2["y"])
train_df2.reset_index(inplace=True)

Създаваме нов модел „Пророк“ и повтаряме цикъла на прогнозиране, който вече направихме по-горе:

In [ ]:
m2 = Prophet()
m2.fit(train_df2)
future2 = m2.make_future_dataframe(periods=prediction_size)
forecast2 = m2.predict(future2)

В този момент трябва да върнем трансформацията на Бокс–Кокс с нашата обратна функция и известната стойност на $\lambda$:


In [ ]:
for column in ["yhat", "yhat_lower", "yhat_upper"]:
    forecast2[column] = inverse_boxcox(forecast2[column], lambda_prophet)

Тук ще използваме повторно нашите инструменти за създаване на рамка от данни за сравнение и изчисляване на грешките:


In [ ]:
cmp_df2 = make_comparison_dataframe(df, forecast2)
for err_name, err_value in calculate_forecast_errors(cmp_df2, prediction_size).items():
    print(err_name, err_value)

Така че определено можем да констатираме повишаване на качеството на модела.

И накрая, нека начертаем нашето предишно представяне с най-новите резултати един до друг. Обърнете внимание, че използваме „prediction_size“ за третия параметър, за да увеличим мащаба на прогнозирания интервал:

In [ ]:
show_forecast(cmp_df, prediction_size, 100, "No transformations")
show_forecast(cmp_df2, prediction_size, 100, "Box–Cox transformation")

Виждаме, че прогнозата за седмичните промени във втората графика е много по-близо до реалните стойности сега.

## 5. Резюме

Разгледахме *Prophet*, библиотека за прогнозиране с отворен код, която е специално насочена към бизнес времеви редове. Ние също направихме някои практически практики в прогнозирането на времеви редове.

Както видяхме, библиотеката на Пророка не прави чудеса и нейните предсказания не са [идеални](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization). Все още зависи от специалиста по данни да проучи прогнозните резултати, да настрои параметрите на модела и да трансформира данните, когато е необходимо.

Тази библиотека обаче е лесна за използване и лесно персонализирана. Единствената възможност да се вземат предвид необичайни дни, които са известни на анализатора предварително, може да има значение в някои случаи.

Като цяло, библиотеката на Пророка си струва да бъде част от вашия аналитичен инструментариум.

## 6. Препратки

- Официално [хранилище на Prophet](https://github.com/facebookincubator/prophet) в GitHub.
- Официална [документация на Prophet] (https://facebookincubator.github.io/prophet/docs/quick_start.html).
- Sean J. Taylor, Benjamin Letham [„Прогнозиране в мащаб“](https://facebookincubator.github.io/prophet/static/prophet_paper_20170113.pdf) — научна статия, обясняваща алгоритъма, който полага основата на „Prophet“.
- [Прогнозиране на трафика на уебсайта с помощта на Facebook's Prophet Library](http://pbpython.com/prophet-overview.html) — Общ преглед на `Prophet` с пример за прогнозиране на трафика на уебсайта.
- Rob J. Hyndman, George Athanasopoulos [„Прогнозиране: принципи и практика“](https://www.otexts.org/fpp) – много добра онлайн книга за прогнозиране на времеви редове.